# Implementacija strokovnega sistema za živali

Primer iz [Učnega načrta za začetnike v umetni inteligenci](http://github.com/microsoft/ai-for-beginners).

V tem primeru bomo implementirali preprost sistem, ki temelji na znanju, za določanje živali na podlagi nekaterih fizičnih značilnosti. Sistem lahko predstavimo z naslednjim AND-OR drevesom (to je del celotnega drevesa, zlahka lahko dodamo še več pravil):

![](../../../../translated_images/AND-OR-Tree.5592d2c70187f283703c8e9c0d69d6a786eb370f4ace67f9a7aae5ada3d260b0.sl.png)


## Naš lasten lupinski sistem strokovnega znanja z vzvratnim sklepanjem

Poskusimo definirati preprost jezik za predstavitev znanja, ki temelji na produkcijskih pravilih. Za definiranje pravil bomo uporabili Python razrede kot ključne besede. Obstajale bodo tri osnovne vrste razredov:
* `Ask` predstavlja vprašanje, ki ga je treba zastaviti uporabniku. Vsebuje nabor možnih odgovorov.
* `If` predstavlja pravilo in je zgolj sintaktični sladkor za shranjevanje vsebine pravila.
* `AND`/`OR` sta razreda za predstavitev AND/OR vej drevesa. Preprosto shranjujeta seznam argumentov znotraj. Za poenostavitev kode je vsa funkcionalnost definirana v nadrejenem razredu `Content`.


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

V našem sistemu bi delovni spomin vseboval seznam **dejstev** kot **pari atribut-vrednost**. Znanstvena baza je lahko opredeljena kot en velik slovar, ki preslika dejanja (nova dejstva, ki bi jih bilo treba vnesti v delovni spomin) v pogoje, izražene kot AND-OR izrazi. Prav tako je mogoče nekatera dejstva `vprašati`.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

Za izvedbo povratnega sklepanja bomo definirali razred `Knowledgebase`. Vsebuje:

* Delovni `spomin` - slovar, ki preslika atribute v vrednosti
* `Pravila` baze znanja v zgoraj določenem formatu

Dve glavni metodi sta:

* `get` za pridobitev vrednosti atributa, pri čemer po potrebi izvede sklepanje. Na primer, `get('color')` pridobi vrednost atributa barve (po potrebi vpraša in shrani vrednost za kasnejšo uporabo v delovnem spominu). Če vprašamo `get('color:blue')`, bo vprašal za barvo in nato vrnil vrednost `y`/`n`, odvisno od barve.
* `eval` izvaja dejansko sklepanje, tj. prehaja skozi AND/OR drevo, ocenjuje podcilje itd.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

Zdaj določimo našo bazo znanja o živalih in izvedimo posvetovanje. Upoštevajte, da vas bo ta klic spraševal vprašanja. Odgovorite lahko z vnosom `y`/`n` za vprašanja z da-ne odgovori ali z navedbo številke (0..N) za vprašanja z daljšimi odgovori z več možnostmi.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## Uporaba PyKnow za naprej sklepanje

V naslednjem primeru bomo poskusili implementirati naprej sklepanje z uporabo ene izmed knjižnic za predstavitev znanja, [PyKnow](https://github.com/buguroo/pyknow/). **PyKnow** je knjižnica za ustvarjanje sistemov za naprej sklepanje v Pythonu, zasnovana tako, da je podobna klasičnemu staremu sistemu [CLIPS](http://www.clipsrules.net/index.html).

Naprej sklepanje bi lahko implementirali tudi sami brez večjih težav, vendar so naivne implementacije običajno manj učinkovite. Za bolj učinkovito ujemanje pravil se uporablja poseben algoritem [Rete](https://en.wikipedia.org/wiki/Rete_algorithm).


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/buguroo/pyknow/

  Cloning https://github.com/buguroo/pyknow/ to /tmp/pip-req-build-3cqeulyl
  Running command git clone --filter=blob:none --quiet https://github.com/buguroo/pyknow/ /tmp/pip-req-build-3cqeulyl
  Resolved https://github.com/buguroo/pyknow/ to commit 48818336f2e9a126f1964f2d8dc22d37ff800fe8
  Preparing metadata (setup.py) ... done
  Using cached frozendict-1.2.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... done
  Using cached schema-0.6.7-py2.py3-none-any.whl (14 kB)
  Created wheel for pyknow: filename=pyknow-1.7.0-py3-none-any.whl size=34228 sha256=b7de5b09292c4007667c72f69b98d5a1b5f7324ff15f9dd8e077c3d5f7aade42
  Stored in directory: /tmp/pip-ephem-wheel-cache-k7jpave7/wheels/81/1a/d3/f6c15dbe1955598a37755215f2a10449e7418500d7bd4b9508
  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3148 sha256=2863d55c240d2409cddf05ccfe600591f8478681549fc97555c47c90dc6bb160
  Stored in directory: /home/rg/.cache/pip/wheels/49/ac/f8/cb8120244e710bdb479c86198b03c7b08c

In [13]:
from pyknow import *
#import pyknow

Naš sistem bomo definirali kot razred, ki podrazreduje `KnowledgeEngine`. Vsako pravilo je definirano z ločeno funkcijo z anotacijo `@Rule`, ki določa, kdaj naj se pravilo sproži. Znotraj pravila lahko dodamo nova dejstva z uporabo funkcije `declare`, dodajanje teh dejstev pa bo povzročilo, da bo sklepni stroj z naprej usmerjenim sklepanjem poklical še nekaj drugih pravil.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

Ko enkrat definiramo bazo znanja, napolnimo delovni spomin z nekaterimi začetnimi dejstvi in nato pokličemo metodo `run()`, da izvedemo sklepanje. Kot rezultat lahko vidite, da so v delovni spomin dodana nova izpeljana dejstva, vključno s končnim dejstvom o živali (če smo pravilno nastavili vsa začetna dejstva).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])


---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za strojno prevajanje [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas opozarjamo, da lahko avtomatizirani prevodi vsebujejo napake ali netočnosti. Izvirni dokument v njegovem izvirnem jeziku je treba obravnavati kot avtoritativni vir. Za ključne informacije priporočamo strokovni prevod s strani človeka. Ne prevzemamo odgovornosti za morebitna nesporazumevanja ali napačne razlage, ki bi izhajale iz uporabe tega prevoda.
